# **Model 2 seq2seq for text Summurization using seq2seq lib**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


### Intro
This is a modification to https://github.com/dongjun-Lee/text-summarization-tensorflow 
I am builging it in a notebook envronment to be able to easily integrate with colab


In [ ]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm() 

  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7413 sha256=cb2189710f3f9008688e890284cc248604f88898489d5d786e4d2a3c1fc3c043
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 12.8 GB  | Proc size: 110.9 MB
GPU RAM Free: 11441MB | Used: 0MB | Util   0% | Total 11441MB


### Utilits

https://github.com/dongjun-Lee/text-summarization-tensorflow/blob/master/utils.py

In [ ]:
!pip install gensim
!pip install wget
  
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 1.x
except Exception:
  pass
import tensorflow as tf

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=ad21089012ff5bb06f3d4782dca7a2a84745c25d57d3a64bcf5be66b493fe029
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
TensorFlow 1.x selected.


In [ ]:
from nltk.tokenize import word_tokenize
import re
import collections
import pickle
import numpy as np
from gensim.models.keyedvectors import KeyedVectors
from gensim.test.utils import get_tmpfile
from gensim.scripts.glove2word2vec import glove2word2vec

#default_path = "/content/drive/My Drive/Colab Notebooks/SCIM/30_testes/"
default_path = "/content/drive/My Drive/Colab Notebooks/"

#train_article_path = "/content/drive/My Drive/Colab Notebooks/sumdata/database_td/pp/resumo.train.pp.txt"
#train_title_path   = "/content/drive/My Drive/Colab Notebooks/sumdata/database_td/pp/titulo.train.pp.txt"
#valid_article_path = "/content/drive/My Drive/Colab Notebooks/sumdata/database_td/pp/resumo.valid.pp.txt"
#valid_title_path   = "/content/drive/My Drive/Colab Notebooks/sumdata/database_td/pp/titulo.valid.pp.txt"


#valid_article_path = "/content/drive/My Drive/Colab Notebooks/sumdata/Database_epia/subgrupo_52.txt"
#valid_title_path   = "/content/drive/My Drive/Colab Notebooks/sumdata/database_td/pp/titulo.valid.pp.txt"

#train_article_path = "/content/drive/My Drive/Colab Notebooks/sumdata/database_scim/pp/train.resumo.txt"
#train_title_path   = "/content/drive/My Drive/Colab Notebooks/sumdata/database_scim/pp/train.titulo.txt"
#valid_article_path = "/content/drive/My Drive/Colab Notebooks/sumdata/database_scim/pp/valid.resumo.txt"
#valid_title_path   = "/content/drive/My Drive/Colab Notebooks/sumdata/database_scim/pp/valid.titulo.txt"

def clean_str(sentence):
    sentence = re.sub("[#.]+", "#", sentence)
    return sentence


def get_text_list(data_path, toy):
    with open (data_path, "r", encoding="utf-8") as f:
        if not toy:
            return [clean_str(x.strip()) for x in f.readlines()][:89351]
        else:
            return [clean_str(x.strip()) for x in f.readlines()][:8306]


def build_dict(step, toy=False):
    if step == "train":
        train_article_list = get_text_list(train_article_path, toy)
        train_title_list = get_text_list(train_title_path, toy)

        words = list()
        for sentence in train_article_list + train_title_list:
            for word in word_tokenize(sentence):
                words.append(word)

        word_counter = collections.Counter(words).most_common()
        word_dict = dict()
        word_dict["<padding>"] = 0
        word_dict["<unk>"] = 1
        word_dict["<s>"] = 2
        word_dict["</s>"] = 3
        for word, _ in word_counter:
            word_dict[word] = len(word_dict)

        with open(default_path + "word_dict.pickle", "wb") as f:
            pickle.dump(word_dict, f)

    elif step == "valid":
        with open(default_path + "word_dict.pickle", "rb") as f:
            word_dict = pickle.load(f)

    reversed_dict = dict(zip(word_dict.values(), word_dict.keys()))

    article_max_len = 150
    summary_max_len = 15
    
    print("\nDictionary Size: " + str(len(word_dict)))
    print("Maximum input size: " + str(article_max_len))
    print("Maximum output abstract size: " + str(summary_max_len) + "\n")

    return word_dict, reversed_dict, article_max_len, summary_max_len


def build_dataset(step, word_dict, article_max_len, summary_max_len, toy=False):
    if step == "train":
        article_list = get_text_list(train_article_path, toy)
        title_list = get_text_list(train_title_path, toy)
    elif step == "valid":
        article_list = get_text_list(valid_article_path, toy)
    else:
        raise NotImplementedError

    x = [word_tokenize(d) for d in article_list]
    x = [[word_dict.get(w, word_dict["<unk>"]) for w in d] for d in x]
    x = [d[:article_max_len] for d in x]
    x = [d + (article_max_len - len(d)) * [word_dict["<padding>"]] for d in x]
    
    if step == "valid":
        return x
    else:        
        y = [word_tokenize(d) for d in title_list]
        y = [[word_dict.get(w, word_dict["<unk>"]) for w in d] for d in y]
        y = [d[:(summary_max_len - 1)] for d in y]
        return x, y


def batch_iter(inputs, outputs, batch_size, num_epochs):
    inputs = np.array(inputs)
    outputs = np.array(outputs)

    num_batches_per_epoch = (len(inputs) - 1) // batch_size + 1
    for epoch in range(num_epochs):
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, len(inputs))
            yield inputs[start_index:end_index], outputs[start_index:end_index]


def get_init_embedding(reversed_dict, embedding_size):
    #glove_file = default_path + "glove/glove.6B.300d.txt"
    #word2vec_file = get_tmpfile(default_path + "word2vec_format.vec")
    #glove2word2vec(glove_file, word2vec_file)
    print("Loading Glove vectors...")
    #word_vectors = KeyedVectors.load_word2vec_format(word2vec_file)

    with open( default_path + "glove/model_glove_300.pkl", 'rb') as handle:
        word_vectors = pickle.load(handle)

        
    word_vec_list = list()
    for _, word in sorted(reversed_dict.items()):
        try:
            word_vec = word_vectors.word_vec(word)
        except KeyError:
            word_vec = np.zeros([embedding_size], dtype=np.float32)

        word_vec_list.append(word_vec)

    # Assign random vector to <s>, </s> token
    word_vec_list[2] = np.random.normal(0, 1, embedding_size)
    word_vec_list[3] = np.random.normal(0, 1, embedding_size)

    return np.array(word_vec_list)


##Model:



Encoder-Decoder model with attention mechanism.
https://github.com/dongjun-Lee/text-summarization-tensorflow/blob/master/model.py

1.   Encoder : Used LSTM cell with stack_bidirectional_dynamic_rnn.
2.   Decoder : Used LSTM BasicDecoder for training, and BeamSearchDecoder for inference.
3.   Attention Mechanism : Used BahdanauAttention with weight normalization.

In [ ]:
import tensorflow as tf
from tensorflow.contrib import rnn
#from utils import get_init_embedding


class Model(object):
    def __init__(self, reversed_dict, article_max_len, summary_max_len, args, forward_only=False):
        self.vocabulary_size = len(reversed_dict)
        self.embedding_size = args.embedding_size
        self.num_hidden = args.num_hidden
        self.num_layers = args.num_layers
        self.learning_rate = args.learning_rate
        self.beam_width = args.beam_width
        if not forward_only:
            self.keep_prob = args.keep_prob
        else:
            self.keep_prob = 1.0
        self.cell = tf.nn.rnn_cell.BasicLSTMCell
        with tf.variable_scope("decoder/projection"):
            self.projection_layer = tf.layers.Dense(self.vocabulary_size, use_bias=False)

        self.batch_size = tf.placeholder(tf.int32, (), name="batch_size")
        self.X = tf.placeholder(tf.int32, [None, article_max_len])
        self.X_len = tf.placeholder(tf.int32, [None])
        self.decoder_input = tf.placeholder(tf.int32, [None, summary_max_len])
        self.decoder_len = tf.placeholder(tf.int32, [None])
        self.decoder_target = tf.placeholder(tf.int32, [None, summary_max_len])
        self.global_step = tf.Variable(0, trainable=False)

        with tf.name_scope("embedding"):
            if not forward_only and args.glove:
                init_embeddings = tf.constant(get_init_embedding(reversed_dict, self.embedding_size), dtype=tf.float32)
            else:
                init_embeddings = tf.random_uniform([self.vocabulary_size, self.embedding_size], -1.0, 1.0)
            self.embeddings = tf.get_variable("embeddings", initializer=init_embeddings)
            self.encoder_emb_inp = tf.transpose(tf.nn.embedding_lookup(self.embeddings, self.X), perm=[1, 0, 2])
            self.decoder_emb_inp = tf.transpose(tf.nn.embedding_lookup(self.embeddings, self.decoder_input), perm=[1, 0, 2])

        with tf.name_scope("encoder"):
            fw_cells = [self.cell(self.num_hidden) for _ in range(self.num_layers)]
            bw_cells = [self.cell(self.num_hidden) for _ in range(self.num_layers)]
            fw_cells = [rnn.DropoutWrapper(cell) for cell in fw_cells]
            bw_cells = [rnn.DropoutWrapper(cell) for cell in bw_cells]

            encoder_outputs, encoder_state_fw, encoder_state_bw = tf.contrib.rnn.stack_bidirectional_dynamic_rnn(
                fw_cells, bw_cells, self.encoder_emb_inp,
                sequence_length=self.X_len, time_major=True, dtype=tf.float32)
            self.encoder_output = tf.concat(encoder_outputs, 2)
            encoder_state_c = tf.concat((encoder_state_fw[0].c, encoder_state_bw[0].c), 1)
            encoder_state_h = tf.concat((encoder_state_fw[0].h, encoder_state_bw[0].h), 1)
            self.encoder_state = rnn.LSTMStateTuple(c=encoder_state_c, h=encoder_state_h)

        with tf.name_scope("decoder"), tf.variable_scope("decoder") as decoder_scope:
            decoder_cell = self.cell(self.num_hidden * 2)

            if not forward_only:
                attention_states = tf.transpose(self.encoder_output, [1, 0, 2])
                attention_mechanism = tf.contrib.seq2seq.BahdanauAttention(
                    self.num_hidden * 2, attention_states, memory_sequence_length=self.X_len, normalize=True)
                decoder_cell = tf.contrib.seq2seq.AttentionWrapper(decoder_cell, attention_mechanism,
                                                                   attention_layer_size=self.num_hidden * 2)
                initial_state = decoder_cell.zero_state(dtype=tf.float32, batch_size=self.batch_size)
                initial_state = initial_state.clone(cell_state=self.encoder_state)
                helper = tf.contrib.seq2seq.TrainingHelper(self.decoder_emb_inp, self.decoder_len, time_major=True)
                decoder = tf.contrib.seq2seq.BasicDecoder(decoder_cell, helper, initial_state)
                outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(decoder, output_time_major=True, scope=decoder_scope)
                self.decoder_output = outputs.rnn_output
                self.logits = tf.transpose(
                    self.projection_layer(self.decoder_output), perm=[1, 0, 2])
                self.logits_reshape = tf.concat(
                    [self.logits, tf.zeros([self.batch_size, summary_max_len - tf.shape(self.logits)[1], self.vocabulary_size])], axis=1)
            else:
                tiled_encoder_output = tf.contrib.seq2seq.tile_batch(
                    tf.transpose(self.encoder_output, perm=[1, 0, 2]), multiplier=self.beam_width)
                tiled_encoder_final_state = tf.contrib.seq2seq.tile_batch(self.encoder_state, multiplier=self.beam_width)
                tiled_seq_len = tf.contrib.seq2seq.tile_batch(self.X_len, multiplier=self.beam_width)
                attention_mechanism = tf.contrib.seq2seq.BahdanauAttention(
                    self.num_hidden * 2, tiled_encoder_output, memory_sequence_length=tiled_seq_len, normalize=True)
                decoder_cell = tf.contrib.seq2seq.AttentionWrapper(decoder_cell, attention_mechanism,
                                                                   attention_layer_size=self.num_hidden * 2)
                initial_state = decoder_cell.zero_state(dtype=tf.float32, batch_size=self.batch_size * self.beam_width)
                initial_state = initial_state.clone(cell_state=tiled_encoder_final_state)
                decoder = tf.contrib.seq2seq.BeamSearchDecoder(
                    cell=decoder_cell,
                    embedding=self.embeddings,
                    start_tokens=tf.fill([self.batch_size], tf.constant(2)),
                    end_token=tf.constant(3),
                    initial_state=initial_state,
                    beam_width=self.beam_width,
                    output_layer=self.projection_layer
                )
                outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(
                    decoder, output_time_major=True, maximum_iterations=summary_max_len, scope=decoder_scope)
                self.prediction = tf.transpose(outputs.predicted_ids, perm=[1, 2, 0])

        with tf.name_scope("loss"):
            if not forward_only:
                crossent = tf.nn.sparse_softmax_cross_entropy_with_logits(
                    logits=self.logits_reshape, labels=self.decoder_target)
                weights = tf.sequence_mask(self.decoder_len, summary_max_len, dtype=tf.float32)
                self.loss = tf.reduce_sum(crossent * weights / tf.to_float(self.batch_size))

                params = tf.trainable_variables()
                gradients = tf.gradients(self.loss, params)
                clipped_gradients, _ = tf.clip_by_global_norm(gradients, 5.0)
                optimizer = tf.train.AdamOptimizer(self.learning_rate)
                self.update = optimizer.apply_gradients(zip(clipped_gradients, params), global_step=self.global_step)

## Test

In [ ]:
import tensorflow as tf
import pickle
from xml.etree import ElementTree
from xml.dom import minidom
from functools import reduce
from xml.etree.ElementTree import Element, SubElement, Comment

class args:
    pass
  
args.num_hidden=150
args.num_layers=2
args.beam_width=10
args.glove="store_true"
args.embedding_size=300

args.learning_rate=1e-3
args.batch_size=64
args.num_epochs=100
args.keep_prob = 0.8

args.toy=True
args.with_model="store_true" 


print("Loading dictionary...")
word_dict, reversed_dict, article_max_len, summary_max_len = build_dict("valid", args.toy)
print("Loading validation dataset...")
valid_x = build_dataset("valid", word_dict, article_max_len, summary_max_len, args.toy)
valid_x_len = [len([y for y in x if y != 0]) for x in valid_x]
print("Loading article and reference...")
article = get_text_list(valid_article_path, args.toy)
reference = get_text_list(valid_title_path, args.toy)

print(len(article))

import time
time_final =[]

for i in range(1):

  tf.reset_default_graph()
  name_model = "model_TD_teste_6" #+str(i+1)
 
  model_checkpoint_path=  "/content/drive/My Drive/Colab Notebooks/TD_models/" +name_model +  "/model.ckpt-139700"

  with tf.Session() as sess:
    print("Loading saved model...")
    model = Model(reversed_dict, article_max_len, summary_max_len, args, forward_only=True)
    saver = tf.train.Saver(tf.global_variables())
    #ckpt = tf.train.get_checkpoint_state("/content/drive/My Drive/Colab Notebooks/TD_models/"+name_model)
    saver.restore(sess, model_checkpoint_path)

    batches = batch_iter(valid_x, [0] * len(valid_x), args.batch_size, 1)

    summary_array = []
    print("Writing summaries to 'result.txt'...")
    start_time = time.time()
    for batch_x, _ in batches:
        batch_x_len = [len([y for y in x if y != 0]) for x in batch_x]

        valid_feed_dict = {
            model.batch_size: len(batch_x),
            model.X: batch_x,
            model.X_len: batch_x_len,
        }
       
        prediction = sess.run(model.prediction, feed_dict=valid_feed_dict)
        prediction_output = [[reversed_dict[y] for y in x] for x in prediction[:, 0, :]]
        with open(default_path + "sumario_sub_52.txt", "a") as f:
            for line in prediction_output:
                summary = list()
                for word in line:
                    if word == "</s>":
                        break
                    if word not in summary:
                        summary.append(word)
                summary_array.append(" ".join(summary))
                print(" ".join(summary), file=f)
    time_final.append(time.time() - start_time)

with open(default_path +"prediction_time.xml", "w+") as f:
  for i in time_final:
    print(str(i) + "\n", file=f)

Loading dictionary...

Dictionary Size: 82668
Maximum input size: 150
Maximum output abstract size: 15

Loading validation dataset...
Loading article and reference...
68
Loading saved model...
INFO:tensorflow:Restoring parameters from /content/drive/My Drive/Colab Notebooks/TD_models/model_TD_teste_6/model.ckpt-139700
Writing summaries to 'result.txt'...


In [ ]:
time_final

[167.3290674686432,
 166.0082564353943,
 166.6884069442749,
 166.2105712890625,
 166.3152334690094,
 167.00601959228516,
 166.96495819091797,
 167.00294637680054,
 166.67039942741394,
 166.46757221221924,
 166.72245979309082,
 166.62844109535217,
 166.5897343158722,
 168.10716795921326,
 169.46397495269775,
 170.80185770988464,
 170.56264662742615,
 170.63388347625732,
 170.43066787719727,
 170.5038914680481,
 170.19576334953308,
 170.17795252799988,
 170.44482731819153,
 170.62771606445312,
 170.06386828422546,
 170.4247133731842,
 170.67617988586426,
 170.8036150932312,
 170.6105010509491,
 170.65170407295227]

[167.3290674686432,
 166.0082564353943,
 166.6884069442749,
 166.2105712890625,
 166.3152334690094,
 167.00601959228516,
 166.96495819091797,
 167.00294637680054,
 166.67039942741394,
 166.46757221221924,
 166.72245979309082,
 166.62844109535217,
 166.5897343158722,
 168.10716795921326,
 169.46397495269775,
 170.80185770988464,
 170.56264662742615,
 170.63388347625732,
 170.43066787719727,
 170.5038914680481,
 170.19576334953308,
 170.17795252799988,
 170.44482731819153,
 170.62771606445312,
 170.06386828422546,
 170.4247133731842,
 170.67617988586426,
 170.8036150932312,
 170.6105010509491,
 170.65170407295227]

In [ ]:
len(time_final)

30

In [ ]:
(28*3600)/ 89351 

1.128135107609316

In [ ]:
import numpy as np
np.mean(time_final)/8306

168.72616659005482

In [ ]:
np.std(time_final)

1.8812367155679042

In [ ]:
import os
import nltk
nltk.download('punkt')
!pip install sumeval
!python -m spacy download en

!git clone https://github.com/google-research/bleurt.git
os.chdir('bleurt')
!pip install .
from bleurt import score
tf.compat.v1.flags.DEFINE_string('f','','')
checkpoint = "bleurt/test_checkpoint"
bleurt = score.BleurtScorer(checkpoint)

!git clone https://github.com/wl-research/nubia.git
os.chdir('nubia')
!pip install -r requirements.txt
from nubia import Nubia
nubia = Nubia()

Streaming output truncated to the last 5000 lines.
Downloading: 94% [775020544 / 815943985] bytes
Downloading: 94% [775028736 / 815943985] bytes
Downloading: 94% [775036928 / 815943985] bytes
Downloading: 94% [775045120 / 815943985] bytes
Downloading: 94% [775053312 / 815943985] bytes
Downloading: 94% [775061504 / 815943985] bytes
Downloading: 94% [775069696 / 815943985] bytes
Downloading: 94% [775077888 / 815943985] bytes
Downloading: 94% [775086080 / 815943985] bytes
Downloading: 94% [775094272 / 815943985] bytes
Downloading: 94% [775102464 / 815943985] bytes
Downloading: 94% [775110656 / 815943985] bytes
Downloading: 94% [775118848 / 815943985] bytes
Downloading: 94% [775127040 / 815943985] bytes
Downloading: 94% [775135232 / 815943985] bytes
Downloading: 94% [775143424 / 815943985] bytes
Downloading: 95% [775151616 / 815943985] bytes
Downloading: 95% [775159808 / 815943985] bytes
Downloading: 95% [775168000 / 815943985] bytes
Downloading: 95% [775176192 / 815943985] bytes
Downloadi

1042301B [00:00, 1849581.89B/s]
456318B [00:00, 1236024.14B/s]


loading archive file pretrained/roBERTa_MNLI
| dictionary: 50264 types


100%|██████████| 548118077/548118077 [00:10<00:00, 53304232.06B/s]


In [ ]:
#https://github.com/chakki-works/sumeval
#https://github.com/Tian312/awesome-text-summarization

from nltk.tokenize import sent_tokenize, word_tokenize
from sumeval.metrics.rouge import RougeCalculator
from nubia import Nubia


def eval(reference_summary,model_summary):

    rouge = RougeCalculator(stopwords=True, lang="en")

    rouge_1 = rouge.rouge_n(
                summary=model_summary,
                references=reference_summary,
                n=1)

    rouge_2 = rouge.rouge_n(
                summary=model_summary,
                references=[reference_summary],
                n=2)
    
    rouge_l = rouge.rouge_l(
                summary=model_summary,
                references=[reference_summary])
    
    nubia_score =  nubia.score(reference_summary, model_summary)

    bleurt_score = scorer.score([reference_summary], [model_summary])

    assert type(bleurt_score) == list and len(bleurt_score) == 1

    return rouge_1, rouge_2,rouge_l, nubia_score, bleurt_score


In [1]:
import cloudpickle

In [ ]:
rouge_1_arr  = []
rouge_2_arr  = []
rouge_L_arr  = []
NUBIA_arr = []
bleurt_arr = []

def prettify(elem):
      """Return a pretty-printed XML string for the Element.
      """
      rough_string = ElementTree.tostring(elem, 'utf-8')
      reparsed = minidom.parseString(rough_string)
      return reparsed.toprettyxml(indent="  ")
  

top = Element('ZakSum')

comment = Comment('Generated by Amr Zaki')
top.append(comment)

i=0
for summ in summary_array:

      
      example = SubElement(top, 'example')
      article_element   = SubElement(example, 'article')
      article_element.text = article[i]
  
      reference_element = SubElement(example, 'reference')
      reference_element.text = reference[i]
  
      summary_element   = SubElement(example, 'summary')
      summary_element.text = summ

      if(len(summ) != 0):
        rouge_1, rouge_2, rouge_L, nubia_score, bleurt_score = eval(reference[i],summ )
      else: 
        rouge_1 = rouge_2 = rouge_L = nubia_score, bleurt_score = 0
  
      eval_element = SubElement(example, 'eval')
      ROUGE_1_element  = SubElement(eval_element, 'ROUGE_1' , {'score':str(rouge_1)})
      ROUGE_2_element  = SubElement(eval_element, 'ROUGE_2' , {'score':str(rouge_2)})
      ROUGE_L_element  = SubElement(eval_element, 'ROUGE_l' , {'score':str(rouge_L)})
      NUBIA_element =  SubElement(eval_element,'NUBIA', {'score':str(nubia_score)})
      BLEURT_element =  SubElement(eval_element,'BLEURT', {'score':str(bleurt_score[0])})
  
      rouge_1_arr.append(rouge_1) 
      rouge_2_arr.append(rouge_2) 
      rouge_L_arr.append(rouge_L)
      NUBIA_arr.append(nubia_score)
      bleurt_arr.append(bleurt_score[0])

      i+=1

top.set('rouge_1', str(np.mean(rouge_1_arr)))
top.set('rouge_2', str(np.mean(rouge_2_arr)))
top.set('rouge_L', str(np.mean(rouge_L_arr)))
top.set('NUBIA', str(np.mean(NUBIA_arr)))
top.set('BLEURT', str(np.mean(bleurt_arr)))


with open(default_path + name_model+".xml", "w+") as f:
    print(prettify(top), file=f)

In [ ]:
import time
start = time.perf_counter()
import tensorflow as tf
import argparse
import pickle
import os

class args:
    pass
  
args.num_hidden=150
args.num_layers=2
args.beam_width=10
args.glove="store_true"
args.embedding_size=300

args.learning_rate=1e-3
args.batch_size=64
args.num_epochs=100-90
args.keep_prob = 0.8

args.toy=False #"store_true"

args.with_model="store_true"

print("Building dictionary...")
word_dict, reversed_dict, article_max_len, summary_max_len = build_dict("train", args.toy)
print("Loading training dataset...")
train_x, train_y = build_dataset("train", word_dict, article_max_len, summary_max_len, args.toy)

list_loss = backup[:]

for i in range(30):

  name_model = "model_TD_teste_" + str(i+29)
  dirpath = "/content/drive/My Drive/Colab Notebooks/"+name_model
  model_checkpoint_path=  "/content/drive/My Drive/Colab Notebooks/TD_models/" +name_model +  "/model.ckpt-139700"

  tf.reset_default_graph()

  with tf.Session() as sess:
    model = Model(reversed_dict, article_max_len, summary_max_len, args)
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver(tf.global_variables())
    saver.restore(sess, model_checkpoint_path )

    batches = batch_iter(train_x, train_y, args.batch_size, args.num_epochs)
    num_batches_per_epoch = (len(train_x) - 1) // args.batch_size + 1

    print("\nIteration starts.")
    print("Number of batches per epoch :", num_batches_per_epoch)
    for batch_x, batch_y in batches:
        batch_x_len = list(map(lambda x: len([y for y in x if y != 0]), batch_x))
        batch_decoder_input = list(map(lambda x: [word_dict["<s>"]] + list(x), batch_y))
        batch_decoder_len = list(map(lambda x: len([y for y in x if y != 0]), batch_decoder_input))
        batch_decoder_output = list(map(lambda x: list(x) + [word_dict["</s>"]], batch_y))

        batch_decoder_input = list(
            map(lambda d: d + (summary_max_len - len(d)) * [word_dict["<padding>"]], batch_decoder_input))
        batch_decoder_output = list(
            map(lambda d: d + (summary_max_len - len(d)) * [word_dict["<padding>"]], batch_decoder_output))

        train_feed_dict = {
            model.batch_size: len(batch_x),
            model.X: batch_x,
            model.X_len: batch_x_len,
            model.decoder_input: batch_decoder_input,
            model.decoder_len: batch_decoder_len,
            model.decoder_target: batch_decoder_output
        }

        _, step, loss = sess.run([model.update, model.global_step, model.loss], feed_dict=train_feed_dict)
        print("step {0}: loss = {1}".format(step, loss))
        list_loss.append(loss)
        break

with open(default_path +"loss.xml", "w+") as f:
  for i in list_loss:
    print(str(i) + "\n", file=f)

Building dictionary...

Dictionary Size: 82668
Maximum input size: 150
Maximum output abstract size: 15

Loading training dataset...
Loading Glove vectors...
INFO:tensorflow:Restoring parameters from /content/drive/My Drive/Colab Notebooks/TD_models/model_TD_teste_29/model.ckpt-139700

Iteration starts.
Number of batches per epoch : 1397
step 139701: loss = 0.3272912800312042
Loading Glove vectors...
INFO:tensorflow:Restoring parameters from /content/drive/My Drive/Colab Notebooks/TD_models/model_TD_teste_30/model.ckpt-139700

Iteration starts.
Number of batches per epoch : 1397
step 139701: loss = 0.5362590551376343
Loading Glove vectors...


NotFoundError: ignored

In [ ]:
backup = list_loss[:]

In [ ]:
with open(default_path +"loss.txt", "w+") as f:
  for i in list_loss:
    print(str(i), file=f)

In [ ]:
import numpy as np
np.mean(list_loss)

0.49355373

In [ ]:
np.std(list_loss)

0.19043596